In [6]:
import duckdb
conn = duckdb.connect('/Users/neifang/Downloads/my_database.db')
conn.execute("CREATE TABLE IF NOT EXISTS inspection AS SELECT * FROM read_csv_auto('/Users/neifang/Downloads/ins.csv', HEADER=TRUE, DELIM=',')")

In [7]:
conn.execute("""
    CREATE TABLE IF NOT EXISTS crime AS 
    SELECT * 
    FROM read_csv_auto(
        '/Users/neifang/Downloads/crime.csv', 
        HEADER=TRUE, 
        DELIM=','
    )
""")

In [8]:
# List all tables in the database
print(conn.execute("SHOW TABLES").fetchall())

[('crime',), ('inspection',)]


In [14]:
conn.execute("DESCRIBE crime;").fetchall()

[('DR_NO', 'BIGINT', 'YES', None, None, None),
 ('Date Rptd', 'TIMESTAMP', 'YES', None, None, None),
 ('DATE OCC', 'TIMESTAMP', 'YES', None, None, None),
 ('TIME OCC', 'BIGINT', 'YES', None, None, None),
 ('AREA', 'BIGINT', 'YES', None, None, None),
 ('AREA NAME', 'VARCHAR', 'YES', None, None, None),
 ('Rpt Dist No', 'BIGINT', 'YES', None, None, None),
 ('Part 1-2', 'BIGINT', 'YES', None, None, None),
 ('Crm Cd', 'BIGINT', 'YES', None, None, None),
 ('Crm Cd Desc', 'VARCHAR', 'YES', None, None, None),
 ('Mocodes', 'VARCHAR', 'YES', None, None, None),
 ('Vict Age', 'BIGINT', 'YES', None, None, None),
 ('Vict Sex', 'VARCHAR', 'YES', None, None, None),
 ('Vict Descent', 'VARCHAR', 'YES', None, None, None),
 ('Premis Cd', 'BIGINT', 'YES', None, None, None),
 ('Premis Desc', 'VARCHAR', 'YES', None, None, None),
 ('Weapon Used Cd', 'BIGINT', 'YES', None, None, None),
 ('Weapon Desc', 'VARCHAR', 'YES', None, None, None),
 ('Status', 'VARCHAR', 'YES', None, None, None),
 ('Status Desc', 'VARCH

In [37]:
conn.execute("SELECT * FROM crime LIMIT 5;").fetchall()

[(190326475,
  datetime.datetime(2020, 3, 1, 0, 0),
  datetime.datetime(2020, 3, 1, 0, 0),
  21,
  7,
  'Wilshire',
  784,
  1,
  510,
  'VEHICLE - STOLEN',
  None,
  0,
  'M',
  'O',
  101,
  'STREET',
  None,
  None,
  'AA',
  'Adult Arrest',
  510,
  998,
  None,
  None,
  '1900 S  LONGWOOD                     AV',
  None,
  34.0375,
  -118.3506,
  'Sunday',
  9652),
 (200106753,
  datetime.datetime(2020, 2, 9, 0, 0),
  datetime.datetime(2020, 2, 8, 0, 0),
  18,
  1,
  'Central',
  182,
  1,
  330,
  'BURGLARY FROM VEHICLE',
  '1822 1402 0344',
  47,
  'M',
  'O',
  128,
  'BUS STOP/LAYOVER (ALSO QUERY 124)',
  None,
  None,
  'IC',
  'Invest Cont',
  330,
  998,
  None,
  None,
  '1000 S  FLOWER                       ST',
  None,
  34.0444,
  -118.2628,
  'Saturday',
  73430),
 (200320258,
  datetime.datetime(2020, 11, 11, 0, 0),
  datetime.datetime(2020, 11, 4, 0, 0),
  17,
  3,
  'Southwest',
  356,
  1,
  480,
  'BIKE - STOLEN',
  '0344 1251',
  19,
  'X',
  'X',
  502,
  'MULTI

In [15]:
existing_columns = conn.execute("DESCRIBE crime").fetchdf()['column_name'].tolist()

# Add column only if it doesn't exist
if 'downtown_distance' not in existing_columns:
    conn.execute("ALTER TABLE crime ADD COLUMN downtown_distance TEXT;")
    print("Column 'downtown_distance' added.")
else:
    print("Column 'downtown_distance' already exists.")

Column 'downtown_distance' added.


In [16]:
#compare the distance between the crime location and downtown LA
distance_query ="""
UPDATE crime
SET downtown_distance = CASE
    WHEN SQRT(
            POW((LAT - 34.0522) * 111, 2) + 
            POW((LON - (-118.2437)) * 92, 2)
        ) <= 10 THEN '0-10km'
    WHEN SQRT(
            POW((LAT - 34.0522) * 111, 2) + 
            POW((LON - (-118.2437)) * 92, 2)
        ) <= 20 THEN '10-20km'
    ELSE '>20km'
END;
"""

In [ ]:
distance_query = """
SELECT 
    *,
    CASE
        WHEN distance_km <= 10 THEN '0-10km'::downtown_distance
        WHEN distance_km <= 20 THEN '10-20km'::downtown_distance
        ELSE '>20km'::downtown_distance
    END AS distance_group
FROM (
    SELECT
        *,
        -- Fast distance approximation from Downtown LA using planar coordinates according to forums:
        -- (111 km/degree for latitude, 92 km/degree for longitude at 34°N)
        SQRT(
            POW((LAT - 34.0522) * 111, 2) + 
            POW((LON - (-118.2437)) * 92, 2)
        ) AS distance_km
    FROM crime
) sub
"""


In [ ]:
conn.execute(distance_query)

In [25]:
conn.execute("DESCRIBE crime").fetchall()

[('DR_NO', 'BIGINT', 'YES', None, None, None),
 ('Date Rptd', 'TIMESTAMP', 'YES', None, None, None),
 ('DATE OCC', 'TIMESTAMP', 'YES', None, None, None),
 ('TIME OCC', 'BIGINT', 'YES', None, None, None),
 ('AREA', 'BIGINT', 'YES', None, None, None),
 ('AREA NAME', 'VARCHAR', 'YES', None, None, None),
 ('Rpt Dist No', 'BIGINT', 'YES', None, None, None),
 ('Part 1-2', 'BIGINT', 'YES', None, None, None),
 ('Crm Cd', 'BIGINT', 'YES', None, None, None),
 ('Crm Cd Desc', 'VARCHAR', 'YES', None, None, None),
 ('Mocodes', 'VARCHAR', 'YES', None, None, None),
 ('Vict Age', 'BIGINT', 'YES', None, None, None),
 ('Vict Sex', 'VARCHAR', 'YES', None, None, None),
 ('Vict Descent', 'VARCHAR', 'YES', None, None, None),
 ('Premis Cd', 'BIGINT', 'YES', None, None, None),
 ('Premis Desc', 'VARCHAR', 'YES', None, None, None),
 ('Weapon Used Cd', 'BIGINT', 'YES', None, None, None),
 ('Weapon Desc', 'VARCHAR', 'YES', None, None, None),
 ('Status', 'VARCHAR', 'YES', None, None, None),
 ('Status Desc', 'VARCH

In [26]:
conn.execute("ALTER TABLE crime ADD COLUMN IF NOT EXISTS nearby_inspection_count INTEGER DEFAULT 0")

In [35]:
# 
conn.execute("""
UPDATE crime 
SET nearby_inspection_count = (
    SELECT COUNT(*)
    FROM inspection
    WHERE 
        -- Bounding box filter (fast first pass)
        inspection.latitude BETWEEN crime.LAT - 0.005 AND crime.LAT + 0.005
        AND inspection.longitude BETWEEN crime.LON - 0.005 AND crime.LON + 0.005
        AND (POW((inspection.latitude - crime.LAT) * 111000, 2) + 
             POW((inspection.longitude - crime.LON) * 92000, 2)) < 250000  
);
""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [26]:
conn.execute("COPY crime TO 'crime_final.csv' (FORMAT CSV, HEADER TRUE);")


In [22]:
conn.close()

In [ ]:
# Verify the update worked by checking a few rows if needed
result = conn.execute("SELECT * FROM crime LIMIT 5").fetchdf()
print(result)

In [ ]:
#helper cell to check file directory
import os

file_path = '/Users/neifang/Downloads/spark_processed/crime_processed.parquet'  # Replace with the actual file path

if os.path.exists(file_path):
    print(f"The path '{file_path}' exists.")
else:
    print(f"The path '{file_path}' does not exist.")

The path '/Users/neifang/Downloads/spark_processed/crime_processed.parquet' exists.
